### Install required pacages


In [1]:
!pip install -U langgraph langchain langchain-openai
!pip install langgraph-supervisor
!pip install pypdf
!pip install lancedb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.37
    Uninstalling langchain-core-0.3.

#### Pass your OpenAI API key and Tavily API key for accessing web search results.



In [15]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj"
os.environ["TAVILY_API_KEY"] = 'tvly-'


We are creating a real-world case study on export-import business compliance, following different guidelines that companies must adhere to. This example will incorporate export policy tools, import policy regulations, and basic guidance.  

To simplify the process, we are using data from the **DGFT Import-Export Policy** available at [DGFT Website](https://www.dgft.gov.in/CP/?opt=itchs-import-export). You can download the necessary data from the provided link.  

Feel free to utilize as many resources as needed to align with the requirements.

In [16]:
import os
import requests

# Define folders and their respective files
folders_and_files = {
    "export_import_policies/how_to_export": {
        "HTE_basics.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/how_to_export/HTE_basics.pdf"
    },
    "export_import_policies/export_product_policy": {
        "export_of_organic_chem.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/export_policy/export_of_organic_chem.pdf",
        "export_of_spices_nuts.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/export_policy/export_of_spices_nuts.pdf",
        "export_of_sugar_n_product.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/export_policy/export_of_sugar_n_prdcut.pdf",
        "export_of_veggine_nuts_cashew.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/export_policy/export_of_veggine_nuts_cashwe.pdf"
    },
    "export_import_policies/import_product_policy": {
        "import_of_veggie2.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/import_policy/import_of_veggie2.pdf",
        "vegetable_nuts_import.pdf": "https://raw.githubusercontent.com/akashAD98/dummy_data/main/Export_import_agent/import_policy/vegitable_nuts_import.pdf"
    }
}

# Function to create folder if not exists and download files
def download_files():
    for folder, files in folders_and_files.items():
        os.makedirs(folder, exist_ok=True)  # Create folder if not exists
        for file_name, url in files.items():
            file_path = os.path.join(folder, file_name)
            if not os.path.exists(file_path):  # Avoid redownloading
                response = requests.get(url)
                if response.status_code == 200:
                    with open(file_path, "wb") as file:
                        file.write(response.content)
                    print(f"Downloaded: {file_name} → {folder}")
                else:
                    print(f"Failed to download: {file_name}")
            else:
                print(f"Already exists: {file_name} → {folder}")

# Run the function
download_files()
print("All files downloaded successfully! ✅")


Downloaded: HTE_basics.pdf → export_import_policies/how_to_export
Downloaded: export_of_organic_chem.pdf → export_import_policies/export_product_policy
Downloaded: export_of_spices_nuts.pdf → export_import_policies/export_product_policy
Downloaded: export_of_sugar_n_product.pdf → export_import_policies/export_product_policy
Downloaded: export_of_veggine_nuts_cashew.pdf → export_import_policies/export_product_policy
Downloaded: import_of_veggie2.pdf → export_import_policies/import_product_policy
Downloaded: vegetable_nuts_import.pdf → export_import_policies/import_product_policy
All files downloaded successfully! ✅


In [17]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from lancedb.rerankers import LinearCombinationReranker
from langchain_core.tools import tool

class DocumentManager:
    def __init__(self, directory_path, table_name, embedding_model="text-embedding-ada-002", reranker_weight=0.3):
        self.directory_path = directory_path
        self.table_name = table_name
        self.embedding_model = embedding_model
        self.reranker_weight = reranker_weight
        self.embeddings = OpenAIEmbeddings(model=embedding_model)
        self.reranker = LinearCombinationReranker(weight=reranker_weight)
        self.docsearch = self._initialize_docsearch()
        print(f"Embeddings for {self.table_name} are done. You can now use it.")

    def _initialize_docsearch(self):
        loader = PyPDFDirectoryLoader(self.directory_path)
        documents = loader.load()
        if not documents:
            print(f"No documents found in directory: {self.directory_path}")
            return None
        else:
            print(f"Loaded {len(documents)} documents from {self.directory_path}")

        text_splitter = CharacterTextSplitter()
        documents = text_splitter.split_documents(documents)
        if not documents:
            print(f"Document splitting resulted in an empty list for directory: {self.directory_path}")
            return None

        return LanceDB.from_documents(
            documents,
            self.embeddings,
            reranker=self.reranker,
            table_name=self.table_name
        )

    def get_docsearch(self):
        if self.docsearch is None:
            raise ValueError(f"Document search initialization failed for table: {self.table_name}")
        return self.docsearch

# Initialize DocumentManagers for different tools
guide_manager = DocumentManager(directory_path="/content/export_import_policies/how_to_export", table_name="guide_export_import")
export_policy_manager = DocumentManager(directory_path="/content/export_import_policies/export_product_policy", table_name="export_policy_info")
import_policy_manager = DocumentManager(directory_path="/content/export_import_policies/import_product_policy", table_name="import_policy_info")


#tools
@tool
def guide_on_how_to_export(query: str):
    """
    Use this tool when the query is about general guidance on how to start an export-import business, documentation requirements, or basic procedures.

    """
    docsearch = guide_manager.get_docsearch()
    docs = docsearch.similarity_search_with_relevance_scores(query, k=5)
    return [{"text": doc.page_content, "score": score} for doc, score in docs]

@tool
def export_policy(query: str):
    """
    Use this tool when the query is related to export policies, regulations, or HS codes for specific products
    """
    docsearch = export_policy_manager.get_docsearch()
    docs = docsearch.similarity_search_with_relevance_scores(query, k=5)
    return [{"text": doc.page_content, "score": score} for doc, score in docs]

@tool
def import_policy(query: str):
    """
    Use this tool when the query is about import regulations, restrictions, and HS codes for any product.
    """
    docsearch = import_policy_manager.get_docsearch()
    docs = docsearch.similarity_search_with_relevance_scores(query, k=5)
    return [{"text": doc.page_content, "score": score} for doc, score in docs]

@tool
def news_related_exim(query: str):
    """
    use this tool when  we need to search latest inforamtion from net such as exporting importing countries .news realted export import .
    """
    tool = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
        include_images=True,

    )
    news = tool.invoke({"query": query})
    return news

Loaded 94 documents from /content/export_import_policies/how_to_export
Embeddings for guide_export_import are done. You can now use it.
Loaded 138 documents from /content/export_import_policies/export_product_policy
Embeddings for export_policy_info are done. You can now use it.
Loaded 32 documents from /content/export_import_policies/import_product_policy
Embeddings for import_policy_info are done. You can now use it.


In [18]:

#Use openai llm . # feel free to use any llm
model = ChatOpenAI(model="gpt-4o", temperature=0)

# List of tools
tools = [guide_on_how_to_export, export_policy,import_policy,news_related_exim]

system_prompt = """You are an expert import export assistant bot. use the avaialble tools for asnwering user quetions. only search the realted prdcut on inofmration if not abibale simply return nithing .try to give best answer to user so he wil get all infromatino"""

memory = MemorySaver()

# Create the agent
graph = create_react_agent(model, tools=tools,prompt=system_prompt, checkpointer=memory)

# Function to print output stream
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

# Example usage:

config = {"configurable": {"thread_id": "thread-1"}}


inputs = {"messages": [("user", "i want to export organic chemicals acyclic aldehyde.")]}
print_stream(graph.stream(inputs,config,stream_mode="values"))


================================ Human Message =================================

i want to export organic chemicals acyclic aldehyde.
================================== Ai Message ==================================
Tool Calls:
  export_policy (call_KMWynrWc5UI7w3IJ5s07kSG8)
 Call ID: call_KMWynrWc5UI7w3IJ5s07kSG8
  Args:
    query: export policies and regulations for organic chemicals acyclic aldehyde
================================= Tool Message =================================
Name: export_policy

[{"text": "Product Description and Export Policy\nItc(hs) Code. Description Export\nPolicy Policy Condition\n2901 Acyclic hydrocarbons.\n29011000 Saturated Free\n29012100 Unsaturated : -- Ethylene Free\n29012200 Unsaturated : -- Propene (propylene) Free\n29012300 Unsaturated : -- Butene (butylene) and isomers thereof Free\n29012400 Unsaturated : -- Buta-1,3-diene and isoprene Free\n290129 Unsaturated : -- Other\n29012910 Acetylene, whether or not in dissolved condition Free\n29012920 Hep

In [ ]:
# we can see that its using memory

In [19]:

config = {"configurable": {"thread_id": "thread-1"}}

inputs = {"messages": [("user", "explain me in more simple points.")]}
print_stream(graph.stream(inputs,config,stream_mode="values"))


================================ Human Message =================================

explain me in more simple points.
================================== Ai Message ==================================

Certainly! Here are the key points for exporting acyclic aldehydes:

1. **No Restrictions**: You can export acyclic aldehydes without any special restrictions or conditions.

2. **HS Code**: The Harmonized System (HS) code for acyclic aldehydes is 29124999. This code is used for classification in international trade.

3. **Documentation**: While there are no specific restrictions, you should still ensure that all standard export documentation is in order, such as invoices, packing lists, and any necessary certificates.

4. **Check Local Regulations**: It's always wise to verify with local export authorities or a trade expert to ensure there are no additional requirements specific to your country or the destination country.

These points should help you understand the basic requirements for e